<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Warning" data-toc-modified-id="Warning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Warning</a></span></li><li><span><a href="#Packages-import" data-toc-modified-id="Packages-import-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Packages import</a></span></li><li><span><a href="#Datasets-selection" data-toc-modified-id="Datasets-selection-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Datasets selection</a></span></li><li><span><a href="#Quantification" data-toc-modified-id="Quantification-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Quantification</a></span><ul class="toc-item"><li><span><a href="#Prepare-fitting-tool" data-toc-modified-id="Prepare-fitting-tool-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Prepare fitting tool</a></span></li><li><span><a href="#Run-fit" data-toc-modified-id="Run-fit-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Run fit</a></span></li></ul></li></ul></div>

# Warning
Compared with the reconstruction, the fitting feature in PASTIS remains quite an adventure and some parts of the code can seem a bit obscure to the user. Sorry ;)

# Packages import

In [ ]:
# packages import
import pastis.fit as fit
import pastis.log as log
import pandas as pd

# that is not porn sorry! It is a file generated automatically by PASTIS containing useful aliases
import pastis.aliases as xxx  

# setting logger level
# from most verbose to very quiet: DEBUG, INFO, WARN, ERROR
log.setLevel(log.ERROR)

# setting up matplotlib
import matplotlib.pylab as plt
%matplotlib notebook


# Datasets selection
Choose what datasets to fit

In [ ]:
# read pkl file containing all the reconstructed signals (see demo_data_processing notebook)
df = pd.read_pickle("db_reconstructed_data.pkl")
df

In [ ]:
# for example, I want to fit all raw datasets acquired with a TE below 30ms
df = df.loc[df["reco_dataset_raw_data_sequence_te"] > 30]

# convert series to dataframe, if needed
if(type(df) is pd.core.series.Series):
    df = df.to_frame().T

df

# Quantification

## Prepare fitting tool

In [ ]:
# create fit object
fit_tool = fit.fit_pastis()

# before the fit, some peak area integration is performed: choose what peak to integrate 
fit_tool.metabolites_area_integration = [xxx.m_NAA_CH3, xxx.m_Cr_CH3, xxx.m_Cho_CH3]
# and their respective integration width (ppm)
fit_tool.area_integration_peak_ranges = [0.1, 0.1, 0.1]

# choose which metabolites to include in the fit basis set
fit_tool.metabolites = [xxx.m_Water,
                        xxx.m_LipA,
                        xxx.m_LipB,
                        xxx.m_LipC,
                        xxx.m_NAA,
                        xxx.m_NAAG,
                        xxx.m_Cr_CH3,
                        xxx.m_Cr_CH2,
                        xxx.m_PCr,
                        xxx.m_GPC,
                        xxx.m_PC,
                        xxx.m_mI,
                        xxx.m_Glu,
                        xxx.m_Gln,
                        xxx.m_Tau]

# --- preparing minimum, maximum and initial fitting parameter sets ---
    
# create default minimum and maximum parameter sets
fit_tool.params_min = fit_tool.params_min.set_default_min().add_macromolecules_min()
fit_tool.params_max = fit_tool.params_max.set_default_max().add_macromolecules_max()

# initial parameter values for fit
fit_tool.params_init = (fit_tool.params_min + fit_tool.params_max) / 2.0

# --- concentrations
    
# fit ranges for concentration for all metabolites and macromolecules
fit_tool.params_min[:, xxx.p_cm] = 0.0
fit_tool.params_max[:, xxx.p_cm] = 200.0
# increase maximum concentration for all macromolecules
fit_tool.params_max[xxx.m_All_MMs, xxx.p_cm] = 1000.0

# set initial concentrations to zero
fit_tool.params_init[:, xxx.p_cm] = 0.0
# start fit with 0.1 concentrations for metabolites in basis set
fit_tool.params_init[fit_tool.metabolites, xxx.p_cm] = 0.1
    
# --- linewidths
    
# minimal damping for all metabolites and macromolecules
fit_tool.params_min[:, xxx.p_dd] = 5

# increase damping ranges for all macromolecules (these are usually broad peaks)
fit_tool.params_min[xxx.m_All_MMs, xxx.p_dd] = 150
fit_tool.params_max[xxx.m_All_MMs, xxx.p_dd] = 300
    
# start fit with minimal linewidth (= narrow peaks)
fit_tool.params_init[:, xxx.p_dd] = fit_tool.params_min[:, xxx.p_dd] * 1.1

# --- frequency shifts

# frequency shift ranges for all metabolites and MMs
fit_tool.params_min[:, xxx.p_df] = -10.0
fit_tool.params_max[:, xxx.p_df] = 10.0
# start fit with no frequency shifts
fit_tool.params_init[:, xxx.p_df] = 0.0

# -- phase shifts
    
# phase shift ranges for all metabolites and MMs
fit_tool.params_min[:, xxx.p_dp] = -0.1
fit_tool.params_max[:, xxx.p_dp] = +0.1
# start fit with no phase shifts
fit_tool.params_init[:, xxx.p_dp] = 0.0

# --- set relations between parameters ---

# first, let's lock all the metabolites
fit_tool.params_linklock[:] = xxx.ll_FIXED

# except the metabolites included in the fit basis set
fit_tool.params_linklock[fit_tool.metabolites, :] = xxx.ll_FREE

# we want a global phase for all metabolites and macromolecules
# in practice we will specify that all phase shifts are obeying the phase of Creatine
fit_tool.params_linklock[fit_tool.metabolites, xxx.p_dp] = xxx.ll_SLAVE1
fit_tool.params_linklock[xxx.m_Cr_CH3, xxx.p_dp] = xxx.ll_MASTER1

# --- an exception for water ---

# all water fitting parameters are free
fit_tool.params_linklock[xxx.m_Water, :] = xxx.ll_FREE

# water max concentration is increased
fit_tool.params_max[xxx.m_Water, xxx.p_cm] = 1000

## Run fit
This following code calls the fit tool prepared above on a set of datasets.

In [ ]:
# list to save fit results
fit_results_list = []

# browse though datasets
for this_index, this_row in df.iterrows():

    # pick reconstructed raw data
    this_data = this_row["reco_dataset_raw_data_obj"]
    
    # remove water residue from data
    this_data = this_data.correct_peak_removal_1d(50, [4.3, 6], False)
    
    # initialize fit tool
    this_fit = fit_tool.copy()
    
    # feed data to the fit tool
    this_fit.data = this_data
        
    # run the fit
    this_fit.run()

    # get the fit results
    this_fit_df = this_fit.to_dataframe()
    
    # store the results
    fit_results_list.append(this_fit_df)

# append all the results and store
df_fit_results = pd.concat(fit_results_list, axis=0)
df_fit_results.to_pickle("db_fit_results.pkl")
